
Montar o drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bibliotecas necessárias

In [ ]:
#!pip install edge-tts
#!pip install PyPDF2
#!pip install pydub
#!pip install moviepy

In [ ]:
from edge_tts import VoicesManager

voices = await VoicesManager.create()
# Lista de vozes em portugues
(voices.find(Language="pt"))#[0]['ShortName']

[{'Name': 'Microsoft Server Speech Text to Speech Voice (pt-BR, ThalitaMultilingualNeural)',
  'ShortName': 'pt-BR-ThalitaMultilingualNeural',
  'Gender': 'Female',
  'Locale': 'pt-BR',
  'SuggestedCodec': 'audio-24khz-48kbitrate-mono-mp3',
  'FriendlyName': 'Microsoft ThalitaMultilingual Online (Natural) - Portuguese (Brazil)',
  'Status': 'GA',
  'VoiceTag': {'ContentCategories': ['General'],
   'VoicePersonalities': ['Friendly', 'Positive']},
  'Language': 'pt'},
 {'Name': 'Microsoft Server Speech Text to Speech Voice (pt-BR, AntonioNeural)',
  'ShortName': 'pt-BR-AntonioNeural',
  'Gender': 'Male',
  'Locale': 'pt-BR',
  'SuggestedCodec': 'audio-24khz-48kbitrate-mono-mp3',
  'FriendlyName': 'Microsoft Antonio Online (Natural) - Portuguese (Brazil)',
  'Status': 'GA',
  'VoiceTag': {'ContentCategories': ['General'],
   'VoicePersonalities': ['Friendly', 'Positive']},
  'Language': 'pt'},
 {'Name': 'Microsoft Server Speech Text to Speech Voice (pt-BR, FranciscaNeural)',
  'ShortName'

### Formatação + PDF para texto

In [1]:
def formatar_texto(texto): # cada arquivo pdf deve ser analisado para uma formatacao especifica
  linhas_texto = texto.splitlines()
  novo_texto = ""
  for i,linha in enumerate(linhas_texto):
    if i > 0:
      if linhas_texto[i-1][-1] == '.': #se o final da ultima linha foi um ponto, esta linha iniciara com uma nova linha
        linha = '\n'+linha
      novo_texto += linha
  return (novo_texto)[len("MeMórias General Vieira da rosa"):]

def pdf_para_texto(arquivo, pagina_inicial, qtd_paginas):
  from PyPDF2 import PdfReader
  reader = PdfReader(arquivo)

  texto = ""
  for i in range(qtd_paginas):
    page = reader.pages[pagina_inicial + i]
    texto_provisorio = page.extract_text()
    texto += formatar_texto(texto_provisorio)

  return texto

### PDF para voz

In [2]:
def livro_para_fala(arquivo, pagina_inicial, qtd_paginas):
  import edge_tts
  from edge_tts import VoicesManager

  texto = pdf_para_texto(arquivo, pagina_inicial, qtd_paginas)
  communicate = edge_tts.Communicate(texto, "pt-BR-AntonioNeural")

  return communicate

Seleção do livro + converção para mp3

In [ ]:
arquivo = "/content/drive/MyDrive/Memorias_Vieira_da_Rosa/memorias-vieira-da-rosa.pdf"
pagina = 175
pagina_pdf = pagina - 2
qtd = 10


fala = livro_para_fala(arquivo, pagina_pdf, qtd)

indice = len(arquivo) - arquivo[::-1].find('/')
diretorio = arquivo[:indice]
nome = f"{pagina}_{pagina+qtd}"

await fala.save(diretorio + nome + ".mp3")
print("concluido")

### MP3 para MP4 c/ imagem

In [ ]:
# link do código:
# https://github.com/horue/Kurippo-MP3-to-MP4-Converter/blob/main/main.py

import moviepy.editor as mp
import os

def converter(audioPath, imagePath):
    nome = str(os.path.basename(audioPath))
    name = os.path.splitext(nome)[0]

    outputPath = os.path.join(os.path.expanduser("~"), f"/content/{name}.mp4")

    audio = mp.AudioFileClip(audioPath, fps=44100)
    video = mp.VideoFileClip(imagePath)


    video_duration = audio.duration


    final_video = video.set_audio(audio).set_duration(video_duration)


    final_video.write_videofile(outputPath, fps=60)



audioPath = "./audio.mp3"
imagePath = "./image.jpg"
converter(audioPath, imagePath)